# 02_Model_Training.ipynb

## Cel

Celem tego notebooka jest:

- Wczytanie i analiza surowych danych.

- Identyfikacja i przetwarzanie brakujących danych w sposób spójny dla zbioru treningowego i testowego.

- Przygotowanie danych do dalszego modelowania poprzez:

   - Imputację braków,

    - Standaryzację cech numerycznych,

    - Kodowanie zmiennych kategorycznych,

- Zapisanie przetworzonych danych do plików CSV, gotowych do użycia w modelach regresji liniowej (z regularyzacją i bez) oraz w sieciach neuronowych.


## Kroki

**1. Wczytanie i przygotowanie danych**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("../Data/train_preprocessed.csv")
test_df = pd.read_csv("../Data/test_preprocessed.csv")

X_train = train_df.drop(columns=["Id", "SalePrice"])  
y_train = train_df["SalePrice"]

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


**2. Trenowanie zwykłej regresji liniowej**

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

lin_reg = LinearRegression()
lin_reg.fit(X_train_split, y_train_split)

y_val_pred = lin_reg.predict(X_val)

r2 = r2_score(y_val, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"Linear Regression R²: {r2}")
print(f"Linear Regression RMSE: {rmse}")


Linear Regression R²: -1.2369387858383782e+17
Linear Regression RMSE: 30802149252450.445


 **Problem: Bardzo złe wyniki**

   - Współczynnik determinacji (R²) wynosił -1.09e+21 (katastrofalny wynik).
   - RMSE (2.89e+15) było ogromne – model przewidywał bardzo źle.
   - Przyczyna: Model miał problem z współliniowością cech i niestabilnymi współczynnikami.

**4. Trenowanie modelu Ridge Regression**

In [3]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1)
ridge_reg.fit(X_train_split, y_train_split)

y_val_pred_ridge = ridge_reg.predict(X_val)

r2_ridge = r2_score(y_val, y_val_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_val, y_val_pred_ridge))

print(f"Ridge Regression R²: {r2_ridge}")
print(f"Ridge Regression RMSE: {rmse_ridge}")


Ridge Regression R²: 0.8672820672510482
Ridge Regression RMSE: 31905.953013943705


**5. Predykcja na zbiorze testowym**

In [4]:
X_test = test_df.drop(columns=["Id"], errors="ignore")

ridge_predictions = ridge_reg.predict(X_test)

ridge_submission_df = pd.DataFrame({
    "Id": test_df["Id"],
    "SalePrice": ridge_predictions
})

ridge_submission_filename = "../Data/ridge_regression_predictions.csv"
ridge_submission_df.to_csv(ridge_submission_filename, index=False)
print(f"Plik submission dla Ridge Regression został zapisany jako: {ridge_submission_filename}")

linear_predictions = lin_reg.predict(X_test)

linear_submission_df = pd.DataFrame({
    "Id": test_df["Id"],
    "SalePrice": linear_predictions
})

linear_submission_filename = "../Data/linear_regression_predictions.csv"
linear_submission_df.to_csv(linear_submission_filename, index=False)
print(f"Plik submission dla Linear Regression został zapisany jako: {linear_submission_filename}")



Plik submission dla Ridge Regression został zapisany jako: ../Data/ridge_regression_predictions.csv
Plik submission dla Linear Regression został zapisany jako: ../Data/linear_regression_predictions.csv


**6. Wyniki z serwisu Kaggle**

- linear_regression_predictions.csv - 1.85690
- ridge_regression_predictions.csv - 0.17252